In [1]:
!pip install pydub
!pip install librosa
!pip install transformers
!pip install accelerate
!pip install torchaudio
!pip install scikit-learn
!pip install datasets

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import shutil
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive')

# 设置源文件夹和目标文件夹路径
src_human_folder = '/content/drive/MyDrive/human'
src_ai_folder = '/content/drive/MyDrive/ai'
dst_folder = '/content/audio_data'

# 创建目标文件夹
os.makedirs(dst_folder, exist_ok=True)

import os
import shutil
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive')

# 设置源文件夹和目标文件夹路径
src_human_folder = '/content/drive/MyDrive/human'
src_ai_folder = '/content/drive/MyDrive/ai'
dst_folder = '/content/audio_data'

# 创建目标文件夹
os.makedirs(dst_folder, exist_ok=True)

# 复制文件夹到目标文件夹
shutil.copytree(src_human_folder, os.path.join(dst_folder, 'human'))
shutil.copytree(src_ai_folder, os.path.join(dst_folder, 'ai'))

# 复制MP3文件到目标文件夹
  #for folder in [src_human_folder, src_ai_folder]:
   # for file in os.listdir(folder):
    #    if file.endswith('.mp3'):
     #       src_path = os.path.join(folder, file)
      #      dst_path = os.path.join(dst_folder, file)
       #     shutil.copy(src_path, dst_path)

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/audio_data/ai'

In [3]:
import librosa
from pydub import AudioSegment
from sklearn.model_selection import train_test_split

# 设置音频参数
target_sr = 16000
max_duration = 10

# 初始化数据列表
audio_data = []
labels = []

# 处理音频文件
for folder in ['human', 'ai']:
    folder_path = os.path.join(dst_folder, folder)
    for file in os.listdir(folder_path):
        if file.endswith('.mp3'):
            audio_path = os.path.join(folder_path, file)

            # 加载音频并转换为WAV格式
            audio = AudioSegment.from_mp3(audio_path)
            audio.export(audio_path.replace('.mp3', '.wav'), format='wav')

            # 读取音频并重采样
            y, sr = librosa.load(audio_path.replace('.mp3', '.wav'), sr=target_sr)

            # 切割音频为10秒以内的片段
            num_segments = int(len(y) / (max_duration * sr)) + 1
            for i in range(num_segments):
                start = i * max_duration * sr
                end = min((i + 1) * max_duration * sr, len(y))
                segment = y[start:end]

                # 添加数据和标注
                audio_data.append(segment)
                labels.append(1 if folder == 'human' else 0)

# 划分训练集和验证集
train_data, val_data, train_labels, val_labels = train_test_split(audio_data, labels, test_size=0.2, random_state=42)

In [4]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import confusion_matrix

# 加载预训练的Wav2Vec2模型和处理器
model = Wav2Vec2ForSequenceClassification.from_pretrained('facebook/wav2vec2-base-960h')
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# 创建数据集
train_dataset = Dataset.from_dict({'audio': train_data, 'label': train_labels})
val_dataset = Dataset.from_dict({'audio': val_data, 'label': val_labels})

# 定义数据预处理函数
def preprocess(batch):
    audio = batch['audio']
    labels = batch['label']

    inputs = processor(audio, sampling_rate=target_sr, return_tensors="pt", padding=True)

    inputs.update({'labels': torch.tensor(labels)})
    return inputs

# 预处理数据集
train_dataset = train_dataset.map(preprocess, batched=True, batch_size=16)
val_dataset = val_dataset.map(preprocess, batched=True, batch_size=16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Map:   0%|          | 0/1118 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import torch
from transformers import TrainerCallback

class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero:
            print(logs)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# 训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy='epoch',
    learning_rate=1e-5,
    output_dir='/content/result',
    logging_dir='/content/logs', # Set the logging directory
    logging_steps=10, # Log every 10 steps
)

# Include the callback in your trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[LoggingCallback()]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.465400,0.420116,0.910714
2,0.254100,0.453528,0.821429


{'loss': 0.6914, 'grad_norm': 0.265536367893219, 'learning_rate': 9.857142857142859e-06, 'epoch': 0.14}
{'loss': 0.6901, 'grad_norm': 0.28720322251319885, 'learning_rate': 9.714285714285715e-06, 'epoch': 0.29}
{'loss': 0.6738, 'grad_norm': 0.8465655446052551, 'learning_rate': 9.571428571428573e-06, 'epoch': 0.43}
{'loss': 0.6574, 'grad_norm': 1.0482221841812134, 'learning_rate': 9.42857142857143e-06, 'epoch': 0.57}
{'loss': 0.5875, 'grad_norm': 2.0417706966400146, 'learning_rate': 9.285714285714288e-06, 'epoch': 0.71}
{'loss': 0.5506, 'grad_norm': 4.492481708526611, 'learning_rate': 9.142857142857144e-06, 'epoch': 0.86}
{'loss': 0.4654, 'grad_norm': 3.3601903915405273, 'learning_rate': 9e-06, 'epoch': 1.0}
{'eval_loss': 0.4201163649559021, 'eval_accuracy': 0.9107142857142857, 'eval_runtime': 33.6221, 'eval_samples_per_second': 8.328, 'eval_steps_per_second': 0.535, 'epoch': 1.0}
{'loss': 0.4324, 'grad_norm': 2.0852510929107666, 'learning_rate': 8.857142857142858e-06, 'epoch': 1.14}
{'l

In [ ]:
# 在验证集上评估模型
predictions = trainer.predict(val_dataset).predictions
predicted_labels = np.argmax(predictions, axis=1)

# 计算混淆矩阵
cm = confusion_matrix(val_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)